In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
df = pd.read_csv('/home/data.csv', index_col=0)
df

,mensaje
msg_id,
1,Para la mejor profesora y nutriologa del mundo...
2,"Los cumpleaÃ±os significan un nuevo comienzo, ..."
3,Te extraÃ±o.
4,"Abuelita, Muchas felicidades te quiero con t..."
5,"Los llevo en todos mis logros conmigo, hoy me ..."
...,...
9996,Chipi tu eres un hermoso sol que ilumina cada ...
9997,Gaby: Muchas Felicidades para mi esposa y madr...
9998,CON MUCHO CARIÃ‘O Y RESPETO PARA LA TIA MAS LINDA


In [ ]:

nltk.download('stopwords')


# Eliminar filas con valores NaN en la columna 'mensaje'
df = df.dropna(subset=['mensaje'])

nltk.download('punkt')
# Obtener las stopwords en español
stop_words = set(stopwords.words('spanish'))



def tokenize_message(text):
    if isinstance(text, str):
        return word_tokenize(text.lower())
    else:
        return []

df['palabras'] = df['mensaje'].apply(tokenize_message)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-5-b7a78da3c8ea>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['palabras'] = df['mensaje'].apply(tokenize_message)


In [ ]:
import numpy as np
from collections import Counter



# Conteo de palabras
words_list = [word for sublist in df['palabras'] for word in sublist]
filtered_words = [word for word in words_list if word.lower() not in stop_words and len(word) > 1]
word_counts = Counter(filtered_words)
most_common_words = word_counts.most_common()

frequencies = word_counts.values()

#Calcular el promedio de las frecuencias
average_frequency = sum(frequencies) / len(frequencies)

# Imprimir el resultado
print(f"El promedio de frecuencia de todas las palabras es: {average_frequency}")

El promedio de frecuencia de todas las palabras es: 9.770855332629356


In [ ]:
print("Palabras más comunes:")
for word, count in most_common_words:
  if count > average_frequency:
    print(f"{word}: {count}")

Palabras más comunes:
amo: 3011
feliz: 2875
cumpleaã±os: 2863
vida: 2162
amor: 2075
siempre: 2020
dã­a: 1936
gracias: 1893
quiero: 1840
mã¡s: 1711
mejor: 1189
aã±o: 1054
â¡feliz: 1019
...: 951
felicidades: 928
ser: 869
espero: 848
hoy: 822
tiempo: 812
muchas: 765
queremos: 760
corazã³n: 721
dios: 720
contigo: 664
deseo: 663
aã±os: 653
nunca: 631
hermosa: 600
mundo: 586
abrazo: 564
cada: 561
hacia: 558
bendiciones: 534
mirar: 531
lado: 521
nuevo: 518
mas: 513
cariã±o: 510
amamos: 508
mamã¡: 504
tan: 451
especial: 431
pronto: 420
si: 410
mando: 382
juntos: 373
mujer: 359
tambiã©n: 356
momento: 338
pases: 328
bien: 328
solo: 324
gran: 323
familia: 322
fuerte: 316
sabes: 316
adelante: 310
dia: 304
dã­as: 300
aniversario: 297
comienzo: 294
gratitud: 291
persona: 290
mami: 285
you: 277
besos: 275
alegrã­a: 273
atrã¡s: 272
cumple: 268
puedo: 266
amiga: 265
flores: 264
significan: 262
tãº: 261
excelente: 261
sonrisa: 258
felicidad: 257
esperanzas: 255
deseamos: 255
soã±ar: 254
mantiene: 254
di

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


# Convertir el conjunto de stopwords a lista
stop_words_list = list(stop_words)

# Vectorización TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words_list)
X_tfidf = tfidf_vectorizer.fit_transform(df['mensaje'])

# Aplicar KMeans para crear categorías
num_clusters = 5  # Número de clusters deseados
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['categoria'] = kmeans.fit_predict(X_tfidf)

# Mostrar las primeras filas del DataFrame con las nuevas categorías
print("\nEjemplos de mensajes con sus categorías asignadas:")
print(df[['mensaje', 'categoria']].head())

# Entrenamiento del clasificador Naive Bayes (usando las categorías creadas)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['categoria'], test_size=0.2, random_state=42)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predicción y evaluación del modelo
y_pred = classifier.predict(X_test)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



Ejemplos de mensajes con sus categorías asignadas:
                                                  mensaje  categoria
msg_id                                                              
1       Para la mejor profesora y nutriologa del mundo...          1
2       Los cumpleaÃ±os significan un nuevo comienzo, ...          4
3                                            Te extraÃ±o.          1
4       Abuelita,   Muchas felicidades te quiero con t...          0
5       Los llevo en todos mis logros conmigo, hoy me ...          1

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.94      0.09      0.16       166
           1       0.75      1.00      0.86      1305
           2       1.00      0.74      0.85        38
           3       0.97      0.29      0.44       380
           4       1.00      0.93      0.96        57

    accuracy                           0.77      1946
   macro avg       0.93      0.61      0.66      1946
weig

<ipython-input-15-67ac5eeee878>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categoria'] = kmeans.fit_predict(X_tfidf)


In [ ]:
df['categoria']

msg_id
1        1
2        4
3        1
4        0
5        1
        ..
9996     1
9997     1
9998     1
9999     1
10000    1
Name: categoria, Length: 9726, dtype: int32